# Show some statistics of the enclosed geometric object with psydat file

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import vstt
from scipy.signal import savgol_filter
from vstt.stats import get_velocity

### Import
A psydat file can be imported using the psychopy `fromFile` function: 
If you want to know the detailed content of the data in psydat file, please check the notebook 'raw_data.ipynb'

In [ ]:
experiment = vstt.Experiment("example.psydat")
stats = experiment.stats

### Plot of results for each trial
For example, a scatter plot of the mouse positions for each trial, labelled by the condition, trial number and repetition number:

In [ ]:
def plot_to_target(ax, group, colors):
    for target_pos, target_radius, positions, color in zip(
        group.target_pos, group.target_radius, group.to_target_mouse_positions, colors
    ):
        ax.plot(positions[:, 0], positions[:, 1], color=color)
        ax.add_patch(
            plt.Circle(
                target_pos,
                target_radius,
                edgecolor="none",
                facecolor=color,
                alpha=0.1,
            )
        )


def plot_to_center(ax, group, colors):
    for central_target_radius, positions, color in zip(
        group.center_radius,
        group.to_center_mouse_positions,
        colors,
    ):
        ax.plot(positions[:, 0], positions[:, 1], color=color)
        ax.add_patch(
            plt.Circle(
                [0, 0],
                central_target_radius,
                edgecolor="none",
                facecolor="black",
                alpha=0.1,
            )
        )


def concatenate(array1, array2):
    return np.concatenate(
        (
            array1,
            array2,
        ),
        axis=0,
    )

In [ ]:
colors = ["blue", "green", "red", "cyan", "magenta", "yellow", "black", "orange"]

nTrials = len(stats["i_trial"].unique())
nReps = len(stats["i_rep"].unique())
fig, axs = plt.subplots(nTrials, nReps, figsize=(6, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    ax = axs[trial, rep]
    ax.set_title(f"[Condition {condition_index}] Trial {trial}, Rep {rep}")
    plot_to_target(ax, group, colors)
    if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
        plot_to_center(ax, group, colors)

plt.show()

### Plot of results for each target
For example, a scatter plot of the mouse positions for each target, labelled by trial number, repetition number, target number and condition:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps * 8, figsize=(6 * 8, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps * 8)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    for positions, color, i in zip(
        group.to_target_mouse_positions, colors, range(len(group))
    ):
        ax = axs[(trial, rep + i)]
        ax.set_title(
            f"[Condition {condition_index}] Trial {trial}, Rep {rep}, Target {i}"
        )
        ax.set_xlim(-0.5, 0.5)
        ax.set_ylim(-0.5, 0.5)
        if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
            positions = concatenate(positions, group.to_center_mouse_positions.iloc[i])
        ax.plot(positions[:, 0], positions[:, 1], color=color)

fig.delaxes(axs[2][6])
fig.delaxes(axs[2][7])
fig.delaxes(axs[3][6])
fig.delaxes(axs[3][7])
plt.show()

### Plot velocity for each trial
For example, a scatter plot of the velocity for each target,displayed in a single plot with velocities shown in the time sequence, labelled by trial number, repetition number and condition:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps, figsize=(6, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    ax = axs[trial, rep]
    ax.set_title(f"[Condition {condition_index}] Trial {trial}, Rep {rep} ")
    for positions, timestamps, color, i in zip(
        group.to_target_mouse_positions,
        group.to_target_timestamps,
        colors,
        range(len(group)),
    ):
        if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
            positions = concatenate(positions, group.to_center_mouse_positions.iloc[i])
            timestamps = concatenate(timestamps, group.to_center_timestamps.iloc[i])
        ax.plot(timestamps[:-1], get_velocity(timestamps, positions), color=color)

plt.show()

For example, a scatter plot of the velocity for each target in separate plot, labelled by trial number, repetition number and condition:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps * 8, figsize=(6 * 8, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps * 8)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    for positions, timestamps, color, i in zip(
        group.to_target_mouse_positions,
        group.to_target_timestamps,
        colors,
        range(len(group)),
    ):
        ax = axs[(trial, rep + i)]
        ax.set_title(
            f"[Condition {condition_index}] Trial {trial}, Rep {rep}, Target {i}"
        )
        if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
            positions = concatenate(positions, group.to_center_mouse_positions.iloc[i])
            timestamps = concatenate(timestamps, group.to_center_timestamps.iloc[i])
        ax.plot(timestamps[:-1], get_velocity(timestamps, positions), color=color)


fig.delaxes(axs[2][6])
fig.delaxes(axs[2][7])
fig.delaxes(axs[3][6])
fig.delaxes(axs[3][7])
plt.show()

For example, a scatter plot of the velocity for each target,displayed in a single plot with velocities starting from the same time point 0,labelled by trial number, repetition number and condition:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps, figsize=(6, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    ax = axs[trial, rep]
    ax.set_title(f"[Condition {condition_index}] Trial {trial}, Rep {rep}")
    for positions, timestamps, color, i in zip(
        group.to_target_mouse_positions,
        group.to_target_timestamps,
        colors,
        range(len(group)),
    ):
        if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
            positions = concatenate(positions, group.to_center_mouse_positions.iloc[i])
            timestamps = concatenate(timestamps, group.to_center_timestamps.iloc[i])
        ax.plot(
            timestamps[:-1] - timestamps[0],
            get_velocity(timestamps, positions),
            color=color,
        )

plt.show()

### apply filter to the mouse positions
For example, a scatter plot of the movement for each target in separate plots, the filtered movement is displayed in black dashed line:


In [ ]:
fig, axs = plt.subplots(nTrials, nReps * 8, figsize=(6 * 8, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps * 8)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    for positions, color, i in zip(
        group.to_target_mouse_positions, colors, range(len(group))
    ):
        ax = axs[(trial, rep + i)]
        ax.set_title(
            f"[Condition {condition_index}] Trial {trial}, Rep {rep}, Target {i}"
        )
        ax.set_xlim(-0.5, 0.5)
        ax.set_ylim(-0.5, 0.5)
        if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
            positions = concatenate(positions, group.to_center_mouse_positions.iloc[i])
        filtered_y = savgol_filter(positions[:, 1], len(positions[:, 0]), 8)
        ax.plot(positions[:, 0], filtered_y, color="black", linestyle="dashed")
        ax.plot(positions[:, 0], positions[:, 1], color=color)

fig.delaxes(axs[2][6])
fig.delaxes(axs[2][7])
fig.delaxes(axs[3][6])
fig.delaxes(axs[3][7])
plt.show()

### apply filter to the mouse position first, then plot the velocity
For example, a scatter plot of the velocity for each target in separate plots, the filtered velocity is displayed in black dashed line:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps * 8, figsize=(6 * 8, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps * 8)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    for positions, timestamps, color, i in zip(
        group.to_target_mouse_positions,
        group.to_target_timestamps,
        colors,
        range(len(group)),
    ):
        ax = axs[(trial, rep + i)]
        ax.set_title(
            f"[Condition {condition_index}] Trial {trial}, Rep {rep}, Target {i}"
        )
        if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
            positions = concatenate(positions, group.to_center_mouse_positions.iloc[i])
            timestamps = concatenate(timestamps, group.to_center_timestamps.iloc[i])
        velocity = get_velocity(timestamps, positions)
        ax.plot(timestamps[:-1], velocity, color=color)
        positions[:, 1] = savgol_filter(positions[:, 1], len(positions[:, 0]), 8)
        velocity_with_filtered_positions = get_velocity(timestamps, positions)
        ax.plot(
            timestamps[:-1],
            velocity_with_filtered_positions,
            color="black",
            linestyle="dashed",
        )

fig.delaxes(axs[2][6])
fig.delaxes(axs[2][7])
fig.delaxes(axs[3][6])
fig.delaxes(axs[3][7])
warnings.filterwarnings("ignore")
plt.show()

### apply filter to the velocity plot to make it smoother
For example, a scatter plot of the velocity for each target in separate plots, the filtered velocity is displayed in black dashed line:

In [ ]:
fig, axs = plt.subplots(nTrials, nReps * 8, figsize=(6 * 8, 6 * nTrials * nReps))
axs = np.reshape(
    axs, (nTrials, nReps * 8)
)  # ensure axs is a 2d-array even if nTrials or nReps is 1
for (trial, rep, condition_index), group in stats.groupby(
    ["i_trial", "i_rep", "condition_index"]
):
    for positions, timestamps, color, i in zip(
        group.to_target_mouse_positions,
        group.to_target_timestamps,
        colors,
        range(len(group)),
    ):
        ax = axs[(trial, rep + i)]
        ax.set_title(
            f"[Condition {condition_index}] Trial {trial}, Rep {rep}, Target {i}"
        )
        if not experiment.trial_list[condition_index]["automove_cursor_to_center"]:
            positions = concatenate(positions, group.to_center_mouse_positions.iloc[i])
            timestamps = concatenate(timestamps, group.to_center_timestamps.iloc[i])
        velocity = get_velocity(timestamps, positions)
        filtered_velocity = savgol_filter(
            velocity, len(timestamps[:-1]), len(timestamps[:-1]) - 1
        )
        ax.plot(timestamps[:-1], filtered_velocity, color="black", linestyle="dashed")
        ax.plot(timestamps[:-1], velocity, color=color)


fig.delaxes(axs[2][6])
fig.delaxes(axs[2][7])
fig.delaxes(axs[3][6])
fig.delaxes(axs[3][7])
warnings.filterwarnings("ignore")
plt.show()